In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
# Import my modules.
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append(str(current_dir) + '/../')

from modules import utils
from modules import models
from modules import preprocess

import importlib
for m in [utils, models, preprocess]:
    importlib.reload(m)

In [3]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 150)

In [4]:
mbd = 'microbusiness_density'

df_train, df_test, df_subm = utils.load_dataset()
df_all, df_census = utils.merge_dataset(df_train, df_test, pop=False, unemploy=False, census=True, coord=True, fix_pop=True, outlier=True, outlier_method='v2')

/tmp/working/code-analysis/../modules/utils.py:293: DtypeWarning: Columns (102) have mixed types.Specify dtype option on import or set low_memory=False.
  df_census = load_census(BASE)


smooth_outlier: max_scale=40
used method: v2
# of fixed cfips: 1210
# of fixed value: 2476


In [5]:
df_all

,cfips,county,state,microbusiness_density,active,year,month,scale,state_i,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,pct_college_2021,pct_foreign_born_2017,pct_foreign_born_2018,pct_foreign_born_2019,pct_foreign_born_2020,pct_foreign_born_2021,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021,lng,lat,SUMLEV,DIVISION,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,NPOPCHG2020,NPOPCHG2021,BIRTHS2020,BIRTHS2021,DEATHS2020,DEATHS2021,NATURALCHG2020,NATURALCHG2021,INTERNATIONALMIG2020,INTERNATIONALMIG2021,DOMESTICMIG2020,DOMESTICMIG2021,NETMIG2020,NETMIG2021,RESIDUAL2020,RESIDUAL2021,GQESTIMATESBASE2020,GQESTIMATES2020,GQESTIMATES2021,RBIRTH2021,RDEATH2021,RNATURALCHG2021,RINTERNATIONALMIG2021,RDOMESTICMIG2021,RNETMIG2021,mbd_origin
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10001_2019-08-01,10001,0,Delaware,5.184650,6624.0,2019,8,0,0,80.8,83.3,85.2,87.0,88.6,14.5,14.6,14.7,15.2,15.6,6.0,5.7,6.1,6.2,6.2,1.3,1.2,1.2,1.2,0.9,57647,58775.0,60910,60117.0,63715.0,-75.568446,39.086391,50,5,181851,182256,184149,405,1893,446,2049,479,2119,-33,-70,5,95,436,1858,441,1953,-3,10,4360,4360,4360,11.184345,11.566436,-0.382091,0.518552,10.141783,10.660335,5.199344
10001_2019-09-01,10001,0,Delaware,5.237020,6672.0,2019,9,1,0,80.8,83.3,85.2,87.0,88.6,14.5,14.6,14.7,15.2,15.6,6.0,5.7,6.1,6.2,6.2,1.3,1.2,1.2,1.2,0.9,57647,58775.0,60910,60117.0,63715.0,-75.568446,39.086391,50,5,181851,182256,184149,405,1893,446,2049,479,2119,-33,-70,5,95,436,1858,441,1953,-3,10,4360,4360,4360,11.184345,11.566436,-0.382091,0.518552,10.141783,10.660335,5.237020
10001_2019-10-01,10001,0,Delaware,5.306093,6760.0,2019,10,2,0,80.8,83.3,85.2,87.0,88.6,14.5,14.6,14.7,15.2,15.6,6.0,5.7,6.1,6.2,6.2,1.3,1.2,1.2,1.2,0.9,57647,58775.0,60910,60117.0,63715.0,-75.568446,39.086391,50,5,181851,182256,184149,405,1893,446,2049,479,2119,-33,-70,5,95,436,1858,441,1953,-3,10,4360,4360,4360,11.184345,11.566436,-0.382091,0.518552,10.141783,10.660335,5.306093
10001_2019-11-01,10001,0,Delaware,5.333566,6795.0,2019,11,3,0,80.8,83.3,85.2,87.0,88.6,14.5,14.6,14.7,15.2,15.6,6.0,5.7,6.1,6.2,6.2,1.3,1.2,1.2,1.2,0.9,57647,58775.0,60910,60117.0,63715.0,-75.568446,39.086391,50,5,181851,182256,184149,405,1893,446,2049,479,2119,-33,-70,5,95,436,1858,441,1953,-3,10,4360,4360,4360,11.184345,11.566436,-0.382091,0.518552,10.141783,10.660335,5.333566
10001_2019-12-01,10001,0,Delaware,5.293535,6744.0,2019,12,4,0,80.8,83.3,85.2,87.0,88.6,14.5,14.6,14.7,15.2,15.6,6.0,5.7,6.1,6.2,6.2,1.3,1.2,1.2,1.2,0.9,57647,58775.0,60910,60117.0,63715.0,-75.568446,39.086391,50,5,181851,182256,184149,405,1893,446,2049,479,2119,-33,-70,5,95,436,1858,441,1953,-3,10,4360,4360,4360,11.184345,11.566436,-0.382091,0.518552,10.141783,10.660335,5.293535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9015_2023-02-01,9015,3134,Connecticut,NaN,NaN,2023,2,42,50,78.7,80.1,83.6,85.7,87.1,14.8,14.9,14.8,13.9,13.9,5.4,5.1,4.9,4.7,4.6,1.9,2.0,2.0,1.8,1.4,62553,64774.0,66550,67365.0,71418.0,-71.987375,41.830066,50,1,116418,116240,116418,-178,178,247,994,322,1262,-75,-268,5,77,-107,349,-102,426,-1,20,4483,4483,4483,8.544731,10.848542,-2.303811,0.661916,3.000112,3.662028,NaN
9015_2023-03-01,9015,3134,Connecticut,NaN,NaN,2023,3,43,50,78.7,80.1,83.6,85.7,87.1,14.8,14.9,14.8,13.9,13.9,5.4,5.1,4.9,4.7,4.6,1.9,2.0,2.0,1.8,1.4,62553,64774.0,66550,67365.0,71418.0,-71.987375,41.830066,50,1,116418,116240,116418,-178,178,247,994,322,1262,-75,-268,5,77,-107,349,-102,426,-1,20,4483,4483,4483,8.544731,10.848542,-2.303811,0.661916,3.000112,3.662028,NaN
9015_2023-04-01,9015,3134,Connecti

In [ ]:
params={
        "act_thre": 140,
        "abs_thre": 0,
        "USE_LAG": 7,
        "USE_TREND": True,
        "blacklist": [],
        "blacklistcfips": [],
        "clip": (-0.0044, 0.0046),
        "model": 'lgb',
        "max_window": 12,
        "light": True
    }

## バリデーション

### 前の予測結果を用いて予測していく

In [25]:
accum_pred = models.LgbmBaseline('params_accum',df_subm, df_all, df_census, start_all_dict=35, params=params)
accum_pred.accum_validation(m_len=2, max_pred_m=1)

add lag features: max_scale=35
created df_all_dict[35]
add lag features: max_scale=36
created df_all_dict[36]
add lag features: max_scale=37
created df_all_dict[37]
add lag features: max_scale=38
created df_all_dict[38]
add lag features: max_scale=39
created df_all_dict[39]
add lag features: max_scale=40
created df_all_dict[40]
valid_times:  40
pred_m:  1
train_times:  39
use df_all_dict[39]
['scale', 'state_i', 'pct_bb_2017', 'pct_bb_2018', 'pct_bb_2019', 'pct_bb_2020', 'pct_bb_2021', 'pct_college_2017', 'pct_college_2018', 'pct_college_2019', 'pct_college_2020', 'pct_college_2021', 'pct_foreign_born_2017', 'pct_foreign_born_2018', 'pct_foreign_born_2019', 'pct_foreign_born_2020', 'pct_foreign_born_2021', 'pct_it_workers_2017', 'pct_it_workers_2018', 'pct_it_workers_2019', 'pct_it_workers_2020', 'pct_it_workers_2021', 'median_hh_inc_2017', 'median_hh_inc_2018', 'median_hh_inc_2019', 'median_hh_inc_2020', 'median_hh_inc_2021', 'lng', 'lat', 'select_rate1_lag1', 'select_rate1_lag2', 'se

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


valid_times:  39
pred_m:  1
train_times:  38
use df_all_dict[38]
use lgb.
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] min_data_in_leaf is set=213, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=213
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] num_iterations is set=30, n_iter=30 will be ignored. Current value: num_iterations=30
valid_times:  38
pred_m:  1
train_times:  37
use df_all_dict[37]
use lgb.
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] min_data_in_leaf is set=213, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=213
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] num_iterations is set=30, n_iter=30 will be ignored. Current value: num_iterations

## Submissionの作成

### 予測結果を活用して予測

In [ ]:
test_subm = models.LgbmBaseline('test_subm_accum', df_subm, df_all, df_census, start_all_dict=40, params=params)
test_subm.create_submission(target_scale=[41, 42, 43, 44], save=False)

In [27]:
df_submission = pd.read_csv('../submission/test_subm_accum_2023-02-25_21:02:46.csv', index_col='row_id')
df_merged = utils.compare_submission(df_submission, 'submission_19346')

In [28]:
df_merged['smape'].mean()

0.009844264000484165

In [29]:
# ensemble
0.01261112205424169

# trend_multi有り
0.03200090765098272

0.03200090765098272

In [52]:
target = 'a' if False else 'b'

In [53]:
target

'b'

In [13]:
dic.get('low_trend_params').get(4).get(params)

AttributeError: 'NoneType' object has no attribute 'get'